In [ ]:
# importing required modules
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "iRSVPred_data.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()

    print('Done!')

In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Flatten,Activation, Dense, GlobalMaxPool2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [ ]:
%load_ext tensorboard
import tensorflow as tf
import datetime, os

In [ ]:
### MODELS
from tensorflow.keras.applications import VGG16

def VGG(seed = None):
    np.random.seed(seed)
    vgg16 = VGG16(weights="imagenet", include_top=False)
    for layer in vgg16.layers[:13]:
        layer.trainable = False
    for layer in vgg16.layers[13:]:
        layer.trainable = True
    y = (vgg16.get_layer("block4_conv3")).output
    mx_y = GlobalMaxPool2D()(y)
    x = BatchNormalization()(mx_y)
    x = Dropout(.5)(x)
    x = Dense(1024, activation='relu', name="dense_1024")(x)
    x = BatchNormalization()(x)
    x = Dropout(.5)(x)
    preds = Dense(11,activation='softmax')(x)
    model = Model(inputs=vgg16.input, outputs=preds)
    return model

In [ ]:
import numpy as np
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

"""LOAD DATA"""
# path to folder with data
path = ""
# expected data structure inside folder: train, test, val. in each folder: one folder for each class,

train_datagen = ImageDataGenerator(featurewise_center=False,
                                   rotation_range = 5, fill_mode="nearest",
                                   zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0, 
                                   horizontal_flip = True, vertical_flip=True,
                                   brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)

validation_set = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)
test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""CREATE & TRAIN MODELS"""

model_vgg = VGG()
epochs = 50
opt = Adam(learning_rate=1e-3)
model_vgg.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
model_vgg.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)

opt = Adam(learning_rate=1e-5)
model_vgg.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)

# save model weights
model_vgg.save_weights("VGG16_weights_basmati_final.h5")
model_vgg.save("VGG16_model_basmati_final.h5")

"""EVALUATE PERFORMANCE ON THE TEST SET"""

y_test = test_set_V2.classes
pred = np.argmax(model_vgg.predict_generator(test_set_V2, steps = test_set_V2.n), axis=1)
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
### MODELS-VGG19

from tensorflow.keras.applications import VGG19

def VGG(seed = None):
    np.random.seed(seed)
    vgg19 = VGG19(weights="imagenet", include_top=False)
    for layer in vgg19.layers[:17]:
        layer.trainable = False
    for layer in vgg19.layers[17:]:
        layer.trainable = True
    y = (vgg19.get_layer("block5_conv2")).output
    mx_y = GlobalMaxPool2D()(y)
    x = BatchNormalization()(mx_y)
    x = Dropout(.5)(x)
    x = Dense(1024, activation='relu', name="dense_1024")(x)
    x = BatchNormalization()(x)
    x = Dropout(.5)(x)
    preds = Dense(11,activation='softmax')(x)
    model = Model(inputs=vgg19.input, outputs=preds)
    return model
import numpy as np
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
"""LOAD DATA"""
# path to folder with data
path = ""
# specify image data generator with data augmentation (train_datagen) resp. without (no_DA_IDG)
train_datagen = ImageDataGenerator(featurewise_center=False,
                                   rotation_range = 5, fill_mode="nearest",
                                   zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0,
                                   horizontal_flip = True, vertical_flip=True,
                                   brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()
# in training set: use data augmentation image data generator, for validation and test: no data augmentation.
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)
validation_set = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)

test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""CREATE & TRAIN MODELS"""
model_vgg = VGG()

epochs = 50
opt = Adam(learning_rate=1e-3)
model_vgg.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
model_vgg.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)

opt = Adam(learning_rate=1e-5)
model_vgg.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)

# save model weights
model_vgg.save_weights("VGG19_weights_basmati_final.h5")
model_vgg.save("VGG19_model_basmati_final.h5")

"""EVALUATE PERFORMANCE ON THE TEST SET"""

y_test = test_set_V2.classes
pred = np.argmax(model_vgg.predict_generator(test_set_V2, steps = test_set_V2.n), axis=1)
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
### MODELS--RESNET50

from tensorflow.keras.applications import ResNet50

def R50(seed = None):
    np.random.seed(seed)
    base_model = ResNet50(weights="imagenet", include_top=False)
    for layer in base_model.layers[:86]:
        layer.trainable = False
    for layer in base_model.layers[86:]:
        layer.trainable = True
    x = (base_model.get_layer("conv4_block1_0_conv")).output
    mx = GlobalMaxPool2D()(x)
    x = BatchNormalization()(mx)
    x = Dropout(.5)(x)
    x = Dense(1024, activation='relu', name="dense_1024")(x)
    x = BatchNormalization()(x)
    x = Dropout(.5)(x)
    preds = Dense(11,activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=preds)
    return model
import numpy as np
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

"""LOAD DATA"""
# path to folder with data
path = ""

train_datagen = ImageDataGenerator(featurewise_center=False,
                                   rotation_range = 5, fill_mode="nearest",
                                   zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0, 
                                   horizontal_flip = True, vertical_flip=True,
                                   brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()

# in training set: use data augmentation image data generator, for validation and test: no data augmentation.
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)

validation_set = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)

test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""CREATE & TRAIN MODELS"""

model_r50 = R50()

epochs = 50
opt = Adam(learning_rate=1e-3)
model_r50.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
model_r50.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)

opt = Adam(learning_rate=1e-5)
model_r50.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)
# save model weights
model_r50.save_weights("R50_weights_basmati_final.h5")
model_r50.save("R50_model_basmati_final.h5")

"""EVALUATE PERFORMANCE ON THE TEST SET"""

y_test = test_set_V2.classes
pred = np.argmax(model_r50.predict_generator(test_set_V2, steps = test_set_V2.n), axis=1)
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
### MODELS-XceptionNet

from tensorflow.keras.applications import Xception

def xcep(seed = None):
    np.random.seed(seed)
    Xcep = Xception(weights="imagenet", include_top=False)
    for layer in Xcep.layers[:36]:
        layer.trainable = False
    for layer in Xcep.layers[36:]:
        layer.trainable = True
    y = (Xcep.get_layer("block5_sepconv1")).output
    mx_y = GlobalMaxPool2D()(y)
    x = BatchNormalization()(mx_y)
    x = Dropout(.5)(x)
    x = Dense(1024, activation='relu', name="dense_1024")(x)
    x = BatchNormalization()(x)
    x = Dropout(.5)(x)
    preds = Dense(11,activation='softmax')(x)
    model = Model(inputs=Xcep.input, outputs=preds)
    return model

import numpy as np
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

"""LOAD DATA"""
# path to folder with data
path = ""

# specify image data generator with data augmentation (train_datagen) resp. without (no_DA_IDG)
train_datagen = ImageDataGenerator(featurewise_center=False,
                                   rotation_range = 5, fill_mode="nearest",
                                   zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0, # occasionally out of range
                                   horizontal_flip = True, vertical_flip=True,
                                   brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()

# in training set: use data augmentation image data generator, for validation and test: no data augmentation.
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224), # typical imagenet dimensions
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)
validation_set =  no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)



test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""CREATE & TRAIN MODELS"""

model_xcep = xcep()


epochs = 50
opt = Adam(learning_rate=1e-3)
model_xcep.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
model_xcep.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)

opt = Adam(learning_rate=1e-5)
model_xcep.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)

# save model weights

model_xcep.save_weights("Xcep_weights_basmati_final.h5")
model_xcep.save("Xcep_model_basmati_final.h5")
"""EVALUATE PERFORMANCE ON THE TEST SET"""

y_test = test_set_V2.classes
pred = np.argmax(model_xcep.predict_generator(test_set_V2, steps = test_set_V2.n), axis=1)
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
### MODELS-effecient_net
from tensorflow.keras.applications import ResNet50, VGG16, VGG19, MobileNetV2, densenet, inception_v3, efficientnet, inception_resnet_v2

# import stuff 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from PIL import Image
from IPython.display import Image, display
import os

from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

# import tensorflow_hub as hub
#from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.applications import ResNet50, VGG16, VGG19, MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input as prepro_res50
from tensorflow.keras.applications.vgg19 import preprocess_input as prepro_vgg19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout, MaxPool2D, MaxPooling2D


import numpy as np
from tensorflow.keras.layers import Dense, GlobalMaxPool2D, BatchNormalization, Dropout
from tensorflow.keras.applications import ResNet50, VGG16, VGG19, MobileNetV2
from tensorflow.keras.models import Model, Sequential

print('Loading EfficientNetB0 ...')

base_model = efficientnet.EfficientNetB0(input_shape=(224, 224, 3),
                   include_top=False,
                   weights='imagenet')

print('EfficientNetB0 loaded')
base_model.trainable = False
num_classes = 11  
x = GlobalMaxPool2D()(base_model.output)
x = BatchNormalization()(x)
x = Dropout(.5)(x)
x = Dense(1024, activation='relu', name="dense_1024")(x)
x = BatchNormalization()(x)
x = Dropout(.5)(x)
preds = Dense(11,activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

# model.summary()



import numpy as np
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

"""LOAD DATA"""
# path to folder with data
path = ""

# specify image data generator with data augmentation (train_datagen) resp. without (no_DA_IDG)
train_datagen = ImageDataGenerator(featurewise_center=False,
                                   rotation_range = 5, fill_mode="nearest",
                                   zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0,
                                   horizontal_flip = True, vertical_flip=True,
                                   brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()

# in training set: use data augmentation image data generator, for validation and test: no data augmentation.
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)

validation_set = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)

test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""CREATE & TRAIN MODELS"""
model_EfficientNetB0 = model

epochs = 50

opt = Adam(learning_rate=1e-3)
model_EfficientNetB0.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
model_EfficientNetB0.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)

opt = Adam(learning_rate=1e-5)
model_EfficientNetB0.fit_generator(generator=training_set, epochs=epochs, validation_data=validation_set, verbose=1)

# save model weights
model_EfficientNetB0.save_weights("EfficientNetB0_weights_basmati_final.h5")
model_EfficientNetB0.save("EfficientNetB0_model_basmati_final.h5")


"""EVALUATE PERFORMANCE ON THE TEST SET"""

y_test = test_set_V2.classes
pred = np.argmax(model_EfficientNetB0.predict_generator(test_set_V2, steps = test_set_V2.n), axis=1)
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
####HDFF model
##Resnet, effecient-net, VGG-16, VGG-19
import numpy as np
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


"""LOAD DATA"""
# path to folder with data
path = ""
train_datagen = ImageDataGenerator(featurewise_center=False,
                                   rotation_range = 5, fill_mode="nearest",
                                   zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0, # occasionally out of range
                                   horizontal_flip = True, vertical_flip=True,
                                   brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()

# in training set: use data augmentation image data generator, for validation and test: no data augmentation.
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224), # typical imagenet dimensions
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)

training_set_V2 =  no_DA_IDG.flow_from_directory(path+"train/", ### TO USE FOR FEATURE EXTRACTION
                                                target_size=(224, 224), # typical imagenet dimensions
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical', shuffle=False)

validation_set = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)

validation_set_V2 = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""Loading Models & their respective weights"""
from tensorflow.keras.models import Sequential, load_model
# save model weights for desired models of your choice
model_vgg = load_model('VGG16_model_basmati_final.h5') 
#model_xcep = load_model('Xcep_model_basmati_final.h5')
model_vgg19 = load_model('VGG19_model_basmati_final.h5') 
model_res50 = load_model('R50_model_basmati_final.h5')
model_effecientNet= load_model('EfficientNetB0_model_basmati_final.h5')

model_vgg.load_weights("VGG16_weights_basmati_final.h5")
model_vgg19.load_weights("VGG19_weights_basmati_final.h5")
#model_xcep.load_weights("Xcep_weights_basmati_final.h5")
model_res50.load_weights("R50_weights_basmati_final.h5")
model_effecientNet.load_weights('EfficientNetB0_weights_basmati_final.h5')

"""MODEL COMBINATION"""

vgg_extractor = Model(inputs=model_vgg.input, outputs=model_vgg.get_layer("dense_1024").output)
#xcep_extractor = Model(inputs=model_xcep.input, outputs=model_xcep.get_layer("dense_1024").output)
vgg19_extractor = Model(inputs=model_vgg19.input, outputs=model_vgg19.get_layer("dense_1024").output)
r50_extractor = Model(inputs=model_res50.input, outputs=model_res50.get_layer("dense_1024").output)
effecientNet_extractor =Model(inputs=model_effecientNet.input, outputs=model_effecientNet.get_layer("dense_1024").output)

y_train = to_categorical(training_set_V2.classes)
X_train_m1 = vgg_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
# X_train_m2 = xcep_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train_m2 = effecientNet_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train_m3 = vgg19_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train_m4 = r50_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
X_train = np.concatenate([X_train_m1, X_train_m2, X_train_m3, X_train_m4], axis=1)
y_val = to_categorical(validation_set_V2.classes)
X_val_m1 = vgg_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
#X_val_m2 = xcep_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m2 = effecientNet_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m3 = vgg19_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m4 = r50_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val = np.concatenate([X_val_m1, X_val_m2, X_val_m3, X_val_m4], axis=1)
y_test = to_categorical(test_set_V2.classes)
X_test_m1 = vgg_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
#X_test_m2 = xcep_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m2 = effecientNet_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m3 = vgg19_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m4 = r50_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test = np.concatenate([X_test_m1, X_test_m2, X_test_m3, X_test_m4], axis=1)

"""TRAIN & TEST FEATURE EXTRACTION MODEL"""
np.random.seed(668)
opt = Adam(learning_rate=1e-3)
model = Sequential()
model.add(Dropout(0.5, input_shape=(4096,)))
model.add(BatchNormalization())
model.add(Dense(11, activation="softmax"))
model1=model

model1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

epochs = 200
history1=model1.fit(X_train, y_train, batch_size=16, epochs=epochs, verbose=2, shuffle=True, validation_data=(X_val, y_val))

"""EVALUATE MODEL ON TEST DATA"""
y_test = np.argmax(y_test, axis=1)
pred = np.argmax(model1.predict(X_test), axis=1)
model1.save('model_vgg16_vgg19_Res50_effecient.h5')
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
from tensorflow.keras.models import Sequential, load_model

In [ ]:
####HDFF model
##Resnet, xception-net, VGG-16, VGG-19

import numpy as np
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


"""LOAD DATA"""
# path to folder with data
path = ""

train_datagen = ImageDataGenerator(featurewise_center=False,
                                   rotation_range = 5, fill_mode="nearest",
                                   zoom_range=[1/1.0, 1/1.0], width_shift_range=0.0, height_shift_range=0.0, # occasionally out of range
                                   horizontal_flip = True, vertical_flip=True,
                                   brightness_range=[0.5, 1.3], channel_shift_range=20)

no_DA_IDG = ImageDataGenerator()

# in training set: use data augmentation image data generator, for validation and test: no data augmentation.
training_set = train_datagen.flow_from_directory(path+"train/",
                                                target_size=(224, 224), # typical imagenet dimensions
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical', shuffle=True)

training_set_V2 =  no_DA_IDG.flow_from_directory(path+"train/", ### TO USE FOR FEATURE EXTRACTION
                                                target_size=(224, 224), # typical imagenet dimensions
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical', shuffle=False)

validation_set = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                shuffle=True)

validation_set_V2 = no_DA_IDG.flow_from_directory(path+"val/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

test_set_V2 = no_DA_IDG.flow_from_directory(path+"test/",
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=1,
                                                class_mode='categorical',
                                                shuffle=False)

###############################################################################################
"""Loading Models & their respective weights"""
from tensorflow.keras.models import Sequential, load_model
# save model weights


model_vgg = load_model('VGG16_model_basmati_final.h5') 
model_xcep = load_model('Xcep_model_basmati_final.h5')
model_vgg19 = load_model('VGG19_model_basmati_final.h5') 
model_res50 = load_model('R50_model_basmati_final.h5')
#model_effecientNet= load_model('EfficientNetB0_model_basmati_final.h5')

model_xcep.load_weights("Xcep_weights_basmati_final.h5")
model_vgg.load_weights("VGG16_weights_basmati_final.h5")
model_res50.load_weights("R50_weights_basmati_final.h5")
model_vgg19.load_weights("VGG19_weights_basmati_final.h5")
#model_effecientNet.load_weights('EfficientNetB0_weights_basmati_final.h5')
print("rakesh1")


"""MODEL COMBINATION"""

vgg_extractor = Model(inputs=model_vgg.input, outputs=model_vgg.get_layer("dense_1024").output)
xcep_extractor = Model(inputs=model_xcep.input, outputs=model_xcep.get_layer("dense_1024").output)
vgg19_extractor = Model(inputs=model_vgg19.input, outputs=model_vgg19.get_layer("dense_1024").output)
r50_extractor = Model(inputs=model_res50.input, outputs=model_res50.get_layer("dense_1024").output)
#effecientNet_extractor =Model(inputs=model_effecientNet.input, outputs=model_effecientNet.get_layer("dense_1024").output)


y_train = to_categorical(training_set_V2.classes)

X_train_m1 = vgg_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)

X_train_m2 = xcep_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
#X_train_m2 = effecientNet_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)

X_train_m3 = vgg19_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)

X_train_m4 = r50_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)

X_train = np.concatenate([X_train_m1, X_train_m2, X_train_m3, X_train_m4], axis=1)

y_val = to_categorical(validation_set_V2.classes)
X_val_m1 = vgg_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m2 = xcep_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
#X_val_m2 = effecientNet_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m3 = vgg19_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val_m4 = r50_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
X_val = np.concatenate([X_val_m1, X_val_m2, X_val_m3, X_val_m4], axis=1)

y_test = to_categorical(test_set_V2.classes)
X_test_m1 = vgg_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m2 = xcep_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
#X_test_m2 = effecientNet_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m3 = vgg19_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test_m4 = r50_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
X_test = np.concatenate([X_test_m1, X_test_m2, X_test_m3, X_test_m4], axis=1)


"""TRAIN & TEST FEATURE EXTRACTION MODEL"""

np.random.seed(668)

opt = Adam(learning_rate=1e-3)
model = Sequential()
model.add(Dropout(0.5, input_shape=(4096,)))
model.add(BatchNormalization())
model.add(Dense(11, activation="softmax"))
model1=model

model1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

epochs = 200
history1=model1.fit(X_train, y_train, batch_size=16, epochs=epochs, verbose=2, shuffle=True, validation_data=(X_val, y_val))

"""EVALUATE MODEL ON TEST DATA"""
y_test = np.argmax(y_test, axis=1)
pred = np.argmax(model1.predict(X_test), axis=1)
model1.save('model_vgg16_vgg19_Res50_xcep.h5')
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))